In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import glob
import pandas as pd

In [3]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyarrow


Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyarrow.parquet import ParquetFile

In [4]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [5]:
import re
results_page = BeautifulSoup(response.content,'lxml')

#### get data from January 2009 through June 2015

In [6]:
def get_taxi_link():
    link_array = []
    pattern = r"(http.*yellow_tripdata_(20[0|1][0-9])-(0?\d*).parquet)"
    # pattern = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet"
    match = re.findall(pattern, str(results_page))
    for i in match:
        if int(i[1]) < 2015:
            link_array.append(i[0])
        elif int(i[1]) == 2015 and int(i[2]) <= 6:
            link_array.append(i[0])
    return link_array

In [7]:
link = get_taxi_link()
link

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-06.pa

In [8]:
pattern = r"yellow.*.parquet"
match = re.search(pattern, link[0])
match.group()

'yellow_tripdata_2015-01.parquet'

In [9]:
import os

if os.path.exists("yellow_tripdata_2009-01.parquet"):
    print("true")
else:
    print("doesn't exist")

true


download data to local folder

In [10]:
import os

pattern = r"yellow.*.parquet"
for taxi_data_url in link:
    match = re.search(pattern, taxi_data_url)
    name_of_file = match.group()
    
    # to make sure we're not re-downloading the same file again
    if not os.path.exists(name_of_file):
        response = requests.get(taxi_data_url, stream=True)
        with open(name_of_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                

In [11]:
link[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet'

#### get all column names to normalize columns

In [ ]:
import os
import re

import pandas as pd
from pyarrow.parquet import ParquetFile

pattern = r"yellow.*.parquet"
parquet_column_list = []

for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
        df = pd.read_parquet(file_name)
        column_name = df.columns.to_list()
        if column_name not in parquet_column_list:
            parquet_column_list.append(column_name)

In [ ]:
parquet_column_list

In [12]:
# we save the parquet column list so that later we don't need to 
parquet_column_list = [['VendorID',
  'tpep_pickup_datetime',
  'tpep_dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'RatecodeID',
  'store_and_fwd_flag',
  'PULocationID',
  'DOLocationID',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount',
  'congestion_surcharge',
  'airport_fee'],
 ['vendor_name',
  'Trip_Pickup_DateTime',
  'Trip_Dropoff_DateTime',
  'Passenger_Count',
  'Trip_Distance',
  'Start_Lon',
  'Start_Lat',
  'Rate_Code',
  'store_and_forward',
  'End_Lon',
  'End_Lat',
  'Payment_Type',
  'Fare_Amt',
  'surcharge',
  'mta_tax',
  'Tip_Amt',
  'Tolls_Amt',
  'Total_Amt'],
 ['vendor_id',
  'pickup_datetime',
  'dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'pickup_longitude',
  'pickup_latitude',
  'rate_code',
  'store_and_fwd_flag',
  'dropoff_longitude',
  'dropoff_latitude',
  'payment_type',
  'fare_amount',
  'surcharge',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'total_amount']]

In [13]:
uber_data = pd.read_csv('uber_rides_sample.csv')
uber_data

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1


In [14]:
# change the format of key to datetime
uber_data['key'] = pd.to_datetime(uber_data['key'])

examine how many rows are there for each year and month

In [15]:
examine_df = pd.concat([uber_data['key'].dt.year, uber_data['key'].dt.month], axis=1)
examine_df.columns = ['year', 'month']

In [16]:
examine_df.groupby(['year', 'month']).size()

year  month
2009  1        2555
      2        2292
      3        2703
      4        2574
      5        2660
               ... 
2015  2        2225
      3        2362
      4        2339
      5        2449
      6        2161
Length: 78, dtype: int64

In [17]:
# number of rows needed for each month
200000/78

2564.102564102564

#### according to the average number of rows in each month each year in the sample uber dataset, we do the following:
- sample 2600 columns from the yellow taxi dataset
- remove unecessary column and normalize the yellow taxi columns by mapping their names to match the uber dataset

In [9]:
import geopandas as gpd
shapefile = gpd.read_file("taxi_zones/taxi_zones.shp")
shapefile

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


In [10]:
shapefile = shapefile.to_crs(4326)
shapefile['longitude'] = shapefile.centroid.x  
shapefile['latitude'] = shapefile.centroid.y
column_to_remove = ['Shape_Leng','Shape_Area','OBJECTID','geometry','zone','borough']
shapefile_new = shapefile.drop(columns=column_to_remove)
shapefile_new

/var/folders/6q/g2p62ptx7bd62l9clms9bnhc0000gn/T/ipykernel_26195/2499202013.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['longitude'] = shapefile.centroid.x
/var/folders/6q/g2p62ptx7bd62l9clms9bnhc0000gn/T/ipykernel_26195/2499202013.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile['latitude'] = shapefile.centroid.y


,LocationID,longitude,latitude
0,1,-74.174000,40.691831
1,2,-73.831299,40.616745
2,3,-73.847422,40.864474
3,4,-73.976968,40.723752
4,5,-74.188484,40.552659
...,...,...,...
258,259,-73.852215,40.897932
259,260,-73.906306,40.744235
260,261,-74.013023,40.709139
261,262,-73.946510,40.775932


In [11]:
shapefile_new.loc[56, 'LocationID'] = 57
shapefile_new.loc[103, 'LocationID'] = 104
shapefile_new.loc[104, 'LocationID'] = 105

In [12]:
shapefile_new

,LocationID,longitude,latitude
0,1,-74.174000,40.691831
1,2,-73.831299,40.616745
2,3,-73.847422,40.864474
3,4,-73.976968,40.723752
4,5,-74.188484,40.552659
...,...,...,...
258,259,-73.852215,40.897932
259,260,-73.906306,40.744235
260,261,-74.013023,40.709139
261,262,-73.946510,40.775932


In [13]:
def one_file(file):
    df = pd.read_parquet(file)
    df = df.merge(shapefile_new, left_on='PULocationID', right_on='LocationID', how='left')
    df = df.rename(columns={'latitude': 'pickup_latitude', 'longitude': 'pickup_longitude'})
    df = df.drop('LocationID', axis=1)
    df = df.merge(shapefile_new, left_on='DOLocationID', right_on='LocationID', how='left')
    df = df.rename(columns={'latitude': 'dropoff_latitude', 'longitude': 'dropoff_longitude'})
    df = df.drop('LocationID', axis=1)
    #change name as uber data
    df = df.rename(columns={'tpep_pickup_datetime':'pickup_datetime', 'Trip_Pickup_DateTime':'pickup_datetime', 'Tip_Amt':'tip_amount','Fare_Amt':'fare_amount', 'Passenger_Count':'passenger_count','Start_Lon':'pickup_longitude','Start_Lat':'pickup_latitude', 'End_Lon':'dropoff_longitude', 'End_Lat':'dropoff_latitude'})
    #drop other column
    df = df[['pickup_datetime','tip_amount','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']]
    # sample 
    df = df.sample(2600)
    return df





In [14]:
file =  '/Users/yangmuyu/final-project/yellow_tripdata_2011-07.parquet'
onedf = one_file(file)
onedf.head()

,pickup_datetime,tip_amount,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
12536808,2011-07-27 15:13:00,2.50,40.791705,-73.973049,40.768615,-73.965635
11373963,2011-07-25 06:26:00,1.50,40.756688,-73.972356,40.736824,-73.984052
53089,2011-07-01 02:30:00,0.00,40.717773,-74.007880,40.727620,-73.985937
9193583,2011-07-20 21:48:11,1.02,40.740439,-73.976495,40.736824,-73.984052
66047,2011-07-01 04:52:00,2.00,40.748428,-73.999917,40.804334,-73.951292


In [15]:
import os
import re


yellow_taxi_data = pd.DataFrame()
pattern = r"yellow.*.parquet"

for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
    df_each = one_file(file_name)
    yellow_taxi_data = pd.concat(df_each)
    
yellow_taxi_data.head()

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [61]:
#def get_center_long(locationid):
   # if locationid in shapefile_new['LocationID'] and locationid not in [57, 104, 105]:
   #     return shapefile_new[shapefile_new['LocationID']==locationid]['longitude'].values[0]

#def get_center_lat(locationid):
    #if locationid in shapefile_new['LocationID'] and locationid not in [57, 104, 105]:
     #   return shapefile_new[shapefile_new['LocationID']==locationid]['latitude'].values[0]


In [ ]:
directory = '/Users/yangmuyu/final-project'
for file in directory:
    df_each = one_file(file)
    yellow_taxi_data = pd.concat(df_each)
yellow_taxi_data.head()

In [68]:
def get_center_long(locationid):
    if locationid in shapefile_new['LocationID']:
        return shapefile_new[shapefile_new['LocationID']==locationid]['longitude'].values[0]

def get_center_lat(locationid):
    if locationid in shapefile_new['LocationID']:
        return shapefile_new[shapefile_new['LocationID']==locationid]['latitude'].values[0]


#### find a file with LocationID instead of longitude to test if the get_center_long functions work or not

In [69]:
import os
import re

pattern = r"yellow.*.parquet"
for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
        df = pd.read_parquet(file_name)
        column_name = df.columns.to_list()
        if parquet_column_list.index(column_name) == 0:
            print(file_name)
        break

yellow_tripdata_2011-07.parquet


In [70]:
import os
import re
yellow_taxi_data = pd.DataFrame()
pattern = r"yellow.*2011-07.parquet"

match = re.match(pattern, file)
pd.read_parquet(file_name)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2011-07-01 00:25:00,2011-07-01 00:33:00,5,2.30,1,None,230,239,2,7.7,0.5,0.5,0.00,0.0,0.0,8.70,None,None
1,2,2011-07-01 00:18:00,2011-07-01 00:20:00,5,0.65,1,None,68,68,1,3.7,0.5,0.5,1.00,0.0,0.0,5.70,None,None
2,2,2011-07-01 00:22:00,2011-07-01 00:38:00,5,4.51,1,None,90,238,1,13.3,0.5,0.5,0.00,0.0,0.0,14.30,None,None
3,2,2011-07-01 00:51:00,2011-07-01 00:55:00,5,1.17,1,None,142,48,1,4.9,0.5,0.5,0.00,0.0,0.0,5.90,None,None
4,1,2011-07-01 00:24:58,2011-07-01 00:29:45,2,1.20,1,N,231,158,1,5.3,0.5,0.5,0.10,0.0,0.0,6.40,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14742556,2,2011-07-31 23:18:00,2011-07-31 23:39:00,6,4.80,1,None,229,223,2,15.3,0.5,0.5,0.00,0.0,0.0,16.30,None,None
14742557,2,2011-07-31 23:52:00,2011-08-01 00:10:00,6,8.33,1,None,138,107,1,20.1,0.5,0.5,5.15,0.0,0.0,26.25,None,None
14742558,1,2011-07-31 23:05:49,2011-07-31 23:09:41,1,0.90,1,N,236,262,2,4.9,0.5,0.5,0.00,0.0,0.0,5.90,None,None
14742559,1,2011-07-31 23:11:07,2011-07-31 23:31:25,4,6.60,1,N,262,211,2,18.5,0.5,0.5,0.00,0.0,0.0,19.50,None,None


In [67]:
import os
import re
yellow_taxi_data = pd.DataFrame()


pattern = r"yellow.*2011-07.parquet"

count = 0
for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
        df = pd.read_parquet(file_name)
        sample_df = df.sample(2600)
        column_name = sample_df.columns.to_list()
        
        if parquet_column_list.index(column_name) == 0:
            sample_df_column_filtered = sample_df.loc[:,['fare_amount',
                                                         'tpep_pickup_datetime',
                                                          'passenger_count',
                                                          'PULocationID',
                                                          'DOLocationID']]
            #'PULocationID', 'DOLocationID' will be transformed to longitude and latitude
            #temp_df = sample_df_column_filteredPULocationID'] != sample_df_column_filtered['DOLocationID']]
            sample_df_column_filtered = sample_df_column_filtered[sample_df_column_filtered['PULocationID']]
            sample_df_column_filtered['pickup_longitude'] = sample_df_column_filtered['PULocationID'].apply(get_center_long)
            sample_df_column_filtered['pickup_latitude'] = sample_df_column_filtered['PULocationID'].apply(get_center_lat)
            sample_df_column_filtered['dropoff_longitude'] = sample_df_column_filtered['DOLocationID'].apply(get_center_long)
            sample_df_column_filtered['dropoff_latitude'] = sample_df_column_filtered['DOLocationID'].apply(get_center_lat)
            
            sample_df_column_filtered = sample_df_column_filtered.dropna()
            sample_df_column_filtered = sample_df_column_filtered.drop(columns=['PULocationID','DOLocationID'])
            
            sample_df_column_filtered.columns = ['fare_amount', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
            yellow_taxi_data = pd.concat([yellow_taxi_data, sample_df_column_filtered])
yellow_taxi_data

KeyError: "None of [Int64Index([148, 239, 231, 239, 246, 138, 236, 138, 234, 132,\n            ...\n            125, 144, 231, 238,  79,  48,  48, 262, 239, 164],\n           dtype='int64', length=2404)] are in the [columns]"

#### After testing it successfully on one file, we apply the function to the bigger function that will ingest every file

In [130]:
yellow_taxi_data = pd.DataFrame()
pattern = r"yellow.*.parquet"

count = 0
for file in os.listdir():
    match = re.match(pattern, file)
    if match:
        file_name = match.group()
        df = pd.read_parquet(file_name)
        sample_df = df.sample(2600)
        column_name = sample_df.columns.to_list()
        
        if parquet_column_list.index(column_name) == 0:
            sample_df_column_filtered = sample_df.loc[:,['fare_amount',
                                                         'tpep_pickup_datetime',
                                                         'passenger_count',
                                                         'PULocationID',
                                                         'DOLocationID']]
            #'PULocationID', 'DOLocationID' will be transformed to longitude and latitude
            sample_df_column_filtered = sample_df_column_filtered[sample_df_column_filtered['PULocationID'] != sample_df_column_filtered['DOLocationID']] 
            sample_df_column_filtered = sample_df_column_filtered[(sample_df_column_filtered['PULocationID']<=263)&(sample_df_column_filtered['DOLocationID']<=263)]
            sample_df_column_filtered['pickup_longitude'] = sample_df_column_filtered['PULocationID'].apply(get_center_long)
            sample_df_column_filtered['pickup_latitude'] = sample_df_column_filtered['PULocationID'].apply(get_center_lat)
            sample_df_column_filtered['dropoff_longitude'] = sample_df_column_filtered['DOLocationID'].apply(get_center_long)
            sample_df_column_filtered['dropoff_latitude'] = sample_df_column_filtered['DOLocationID'].apply(get_center_lat)
            sample_df_column_filtered = sample_df_column_filtered.drop(columns=['PULocationID','DOLocationID'])
            sample_df_column_filtered.columns = ['fare_amount', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
        elif parquet_column_list.index(column_name) == 1:
            sample_df_column_filtered = sample_df.loc[:,[ 'Trip_Pickup_DateTime',            
                                                          'Passenger_Count',            
                                                          'Start_Lon',
                                                          'Start_Lat',              
                                                          'End_Lon',
                                                          'End_Lat',
                                                          'Fare_Amt']]
            sample_df_column_filtered.columns = ['pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount']
        elif parquet_column_list.index(column_name) == 2:
            sample_df_column_filtered = sample_df.loc[:,[ 'pickup_datetime',  
                                                          'passenger_count',
                                                          'pickup_longitude',
                                                          'pickup_latitude',
                                                          'dropoff_longitude',
                                                          'dropoff_latitude',  
                                                          'fare_amount']]
        yellow_taxi_data = pd.concat([yellow_taxi_data, sample_df_column_filtered])
        count += 1

In [132]:
yellow_taxi_data[yellow_taxi_data['pickup_longitude']==yellow_taxi_data['pickup_latitude']]

,fare_amount,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
13600791,4.1,2009-10-29 18:38:00,1,0.0,0.0,0.0,0.0
3010124,3.7,2009-10-27 09:15:00,1,0.0,0.0,0.0,0.0
15387350,7.3,2009-10-18 17:47:00,1,0.0,0.0,0.0,0.0
11135901,8.9,2009-10-16 18:52:00,1,0.0,0.0,0.0,0.0
11669366,5.3,2009-10-08 21:33:29,1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
9001991,12.1,2009-05-03 19:51:52,1,0.0,0.0,0.0,0.0
11147221,7.7,2009-05-22 22:39:52,2,0.0,0.0,0.0,0.0
6988018,8.5,2009-05-03 14:00:07,1,0.0,0.0,0.0,0.0
14749366,8.5,2009-05-13 08:56:00,3,0.0,0.0,0.0,0.0


according tho the above dataframe filtering, there're rows where pickup longitude and latitude are the same. Therefore, we will remove these in the final dataset

In [140]:
yellow_taxi_data = yellow_taxi_data[yellow_taxi_data['pickup_longitude']!=yellow_taxi_data['pickup_latitude']]
yellow_taxi_data

""


In [114]:
yellow_taxi_data

,fare_amount,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
5671163,5.3,2011-07-13 19:11:56,1,-73.990458,40.740337,-73.999917,40.748428
3023654,5.7,2011-07-08 09:35:00,1,NaN,NaN,-73.957012,40.780436
830001,4.5,2011-07-02 22:52:00,1,-74.002875,40.734576,-73.990458,40.740337
3679350,4.5,2011-07-09 15:36:00,1,-73.954739,40.765484,-73.949952,40.761900
5129810,3.7,2011-07-12 18:52:02,1,-73.981532,40.773633,-73.978632,40.783961
...,...,...,...,...,...,...,...
14091397,8.9,2009-05-26 14:23:03,1,-74.006757,40.750462,-73.993091,40.737458
8332974,5.7,2009-05-27 02:54:41,1,-73.978316,40.753523,-73.966497,40.754609
11321068,4.5,2009-05-16 17:48:33,2,-74.001299,40.731182,-73.993529,40.744904
3428574,6.5,2009-05-04 21:20:00,5,0.000000,0.000000,0.000000,0.000000


In [39]:
import numpy as np
#array = np.array()
np.concatenate((np.array(column_name), np.array(column_name)))

array(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID',
       'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee', 'VendorID',
       'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'airport_fee'], dtype=object)

In [40]:
filename = "yellow_tripdata_2009-01.parquet"
df = pd.read_parquet(link[0])

In [41]:
index = df.columns

In [42]:
np.array(index)

array(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID',
       'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee'], dtype=object)

#### calculating distance

According to the shapefile "taxi_zones.shp" above, the highest objectID, which is the pickup location ID and drop off location ID, is 263. Therefore, in the Cleaning & filtering step, we remove the IDs that are greater than 263.